In [2]:
import jax
import jax.numpy as jnp
import flax
from flax import nnx
from models import TestConvNet
from utils import clipping_ste

%load_ext autoreload
%autoreload 2

In [2]:
t1 = TestConvNet(rngs=nnx.Rngs(0))
nnx.display(t1)

In [5]:
x = jnp.arange(0., 1.5, 0.01)
rngs = nnx.Rngs(activation=0)
y = clipping_ste(x, 0.0, 1.0, rngs.activation())
print(y)

[1. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 1. 0. 1. 1. 1. 1. 0. 1. 1. 0. 0. 1. 1.
 1. 0. 1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 0. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 0. 1. 0. 0. 1. 1. 0. 0. 0. 0.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 0. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1.
 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1.]
